In [5]:
# 主要ライブラリのimport
import pandas as pd
import numpy as np

#ファイル出力時にパスを指定する際によく使う
import os
import pathlib

# ailab_tools → ailabで作っている各種ツール群
# impala、GSheet、GMail等の操作が簡単にできる
# ドキュメントは下記
# https://s3-ap-northeast-1.amazonaws.com/ailab-smn-valis/ailab-tools/docs/html/index.html

from ailab_tools.smn import ImpalaResource
from ailab_tools.utils import GMail, GSheet

#グラフをnotebook上で描画するためのおまじない
%matplotlib inline

INFO:numexpr.utils:Note: NumExpr detected 32 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/opt/anaconda3/lib/python3.8/site-packages/ailab_tools/utils/pandas_utils.py:118: FutureWarning: The SparseDataFrame class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  sep: str = ",", binary: bool = True) -> pd.SparseDataFrame:


In [2]:
# Impala接続情報 hc5
ircfg = {
    "hosts": ['172.16.60.117'],
    "port": 21050,
    "user": 'vmspool',
    "request_pool": 'adhoc_dm01_pool',
    "httpfs_host": '172.16.60.42',
    "httpfs_port": 14000
}

In [3]:
start_at = "2025/03/01"
end_at = "2025/03/31"

In [6]:
query = (
f"""
with imp as(select
case when user_agent_os_family in('iOS') and is_app=true then 'iOSアプリ'
when user_agent_os_family in('Android') and is_app=true then 'Androidアプリ'
when user_agent_os_family in('iOS') and is_app=false then 'iOSウェブ'
when user_agent_os_family in('Android') and is_app=false then 'Androidウェブ'
else 'PC' end kbn,
concat_ws('/', year, month, day) sum_date,
ssp_id parent_ssp_id, 
ads.child_ssp_id,
case when creative_type in(5,6) then 1 else 0 end nativeflg,
case when creative_type in(10) then 1 else 0 end videoflg,
sum(case when oem_id in(100,200) then net_spend else 0 end)/1000000 sales, 
sum(case when oem_id in(100,200) then imp else 0 end) imp, 
sum(case when ad_type in(0,4) and oem_id in(100,200) then imp else 0 end) rta_imp,
sum((case when (ad_type in(0,4) and oem_id in(100,200)) then net_spend else 0 end))/1000000 rta_sales,
sum(case when oem_id in(100,200) then ctv1 else 0 end) ctv1,
sum(case when oem_id in(100,200) then ctv2 else 0 end) ctv2, 
sum(case when ad_type in(0,4) and oem_id in(100,200) then ctv1 else 0 end) rta_ctv1,  
sum(case when ad_type in(0,4) and oem_id in(100,200) then ctv2 else 0 end) rta_ctv2, 
sum(case when oem_id in(300)then imp else 0 end) cpx_imp,
sum((case when oem_id in(300) then net_spend else 0 end))/1000000 cpx_sales,
sum(case when oem_id in(300)then ctv1 else 0 end) cpx_ctv1,
sum(case when oem_id in(300)then ctv2 else 0 end) cpx_ctv2
from dm.domain_ads ads 
inner join dm.hierarchies hie using(target_id) 
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
where concat_ws('/', year, month ,day) between '{start_at}' and '{end_at}' 
group by 1,2,3,4,5,6
having sum(imp) > 0),
res as(
select 
case when user_agent_os_family in('iOS') and is_app=true then 'iOSアプリ'
when user_agent_os_family in('Android') and is_app=true then 'Androidアプリ'
when user_agent_os_family in('iOS') and is_app=false then 'iOSウェブ'
when user_agent_os_family in('Android') and is_app=false then 'Androidウェブ'
else 'PC' end kbn,
concat_ws('/', year, month, day) sum_date,
ssp_id parent_ssp_id, 
child_ssp_id,
cast(native_request as int) nativeflg, 
cast(video_request as int) videoflg,
sum(case when res.tactics_id > 0 and oem_id in(100,200) then res else 0 end) res, 
sum(case when ad_type = 0 and res.tactics_id > 0 and oem_id in(100,200) then res else 0 end) rta_res, 
sum(case when res.tactics_id > 0 and oem_id=300 then res else 0 end) cpx_res 
from dm.domain_res res 
left join dm.hierarchies b using (target_id) 
where concat_ws('/', res.year, res.month ,res.day) between '{start_at}' and '{end_at}' 
and res.ssp_id > 0 
and res.tactics_id > 0 
group by 1,2,3,4,5,6)
select
case when user_agent_os_family in('iOS') and is_app=true then 'iOSアプリ'
when user_agent_os_family in('Android') and is_app=true then 'Androidアプリ'
when user_agent_os_family in('iOS') and is_app=false then 'iOSウェブ'
when user_agent_os_family in('Android') and is_app=false then 'Androidウェブ'
else 'PC' end kbn,
concat_ws('/', year, month, day) sum_date,
ssp_id parent_ssp_id, 
ads.child_ssp_id,
ads.child_ssp_id display_child_ssp_id,
cast(native_request as int) nativeflg, 
cast(video_request as int) videoflg,
sum(logicad_request_id_1_sum_native_plcmtcnt) req,  
sales, 
res,
imp, 
rta_res,
rta_imp,
rta_sales,
ctv1,
ctv2, 
rta_ctv1,  
rta_ctv2, 
cpx_res,
cpx_imp,
cpx_sales,
cpx_ctv1,
cpx_ctv2
from dm.domain_req ads 
left join res on 
case when ads.user_agent_os_family in('iOS') and ads.is_app=true then 'iOSアプリ'
when ads.user_agent_os_family in('Android') and ads.is_app=true then 'Androidアプリ'
when ads.user_agent_os_family in('iOS') and ads.is_app=false then 'iOSウェブ'
when ads.user_agent_os_family in('Android') and ads.is_app=false then 'Androidウェブ'
else 'PC' end=res.kbn 
and concat_ws('/', ads.year, ads.month, ads.day)=res.sum_date
and ads.ssp_id=res.parent_ssp_id
and ads.child_ssp_id=res.child_ssp_id
and cast(native_request as int)=res.nativeflg
and cast(video_request as int)=res.videoflg
left join imp on 
case when ads.user_agent_os_family in('iOS') and ads.is_app=true then 'iOSアプリ'
when ads.user_agent_os_family in('Android') and ads.is_app=true then 'Androidアプリ'
when ads.user_agent_os_family in('iOS') and ads.is_app=false then 'iOSウェブ'
when ads.user_agent_os_family in('Android') and ads.is_app=false then 'Androidウェブ'
else 'PC' end=imp.kbn 
and concat_ws('/', ads.year, ads.month, ads.day)=imp.sum_date
and ads.ssp_id=imp.parent_ssp_id
and ads.child_ssp_id=imp.child_ssp_id
and cast(native_request as int)=imp.nativeflg
and cast(video_request as int)=imp.videoflg
where concat_ws('/', year, month ,day) between '{start_at}' and '{end_at}' 
and ads.ssp_id>0
group by 1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
"""
)
with ImpalaResource(**ircfg) as ir:
    data_ssp_d = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [7]:
query = (
f"""
with imp as(select
concat_ws('/', year, month, day) sum_date,
ssp_id parent_ssp_id, 
ads.child_ssp_id,
case when creative_type in(5,6) then 1 else 0 end nativeflg,
case when creative_type in(10) then 1 else 0 end videoflg,
sum(case when oem_id in(100,200) then net_spend else 0 end)/1000000 sales, 
sum(case when oem_id in(100,200) then imp else 0 end) imp, 
sum(case when ad_type in(0,4) and oem_id in(100,200) then imp else 0 end) rta_imp,
sum((case when (ad_type in(0,4) and oem_id in(100,200)) then net_spend else 0 end))/1000000 rta_sales,
sum(case when oem_id in(100,200) then ctv1 else 0 end) ctv1,
sum(case when oem_id in(100,200) then ctv2 else 0 end) ctv2, 
sum(case when ad_type in(0,4) and oem_id in(100,200) then ctv1 else 0 end) rta_ctv1,  
sum(case when ad_type in(0,4) and oem_id in(100,200) then ctv2 else 0 end) rta_ctv2, 
sum(case when oem_id in(300)then imp else 0 end) cpx_imp,
sum((case when oem_id in(300) then net_spend else 0 end))/1000000 cpx_sales,
sum(case when oem_id in(300)then ctv1 else 0 end) cpx_ctv1,
sum(case when oem_id in(300)then ctv2 else 0 end) cpx_ctv2
from dm.domain_ads ads 
inner join dm.hierarchies hie using(target_id) 
inner join agency_console.campaign cp on ads.strategy_id = cp.real_strategy_id 
inner join agency_console.campaign_agency_margin cam on cp.campaign_id = cam.campaign_id and concat_ws('-', year, month, day) between cam.start_date and cam.end_date
inner join agency_console.campaign_smn_margin csm on cp.campaign_id = csm.campaign_id and concat_ws('-', year, month, day) between csm.start_date and csm.end_date 
where concat_ws('/', year, month ,day) between '{start_at}' and '{end_at}' 
group by 1,2,3,4,5
having sum(imp) > 0),
res as(
select 
concat_ws('/', year, month, day) sum_date,
ssp_id parent_ssp_id, 
child_ssp_id,
cast(native_request as int) nativeflg, 
cast(video_request as int) videoflg,
sum(case when res.tactics_id > 0 and oem_id in(100,200) then res else 0 end) res, 
sum(case when ad_type = 0 and res.tactics_id > 0 and oem_id in(100,200) then res else 0 end) rta_res, 
sum(case when res.tactics_id > 0 and oem_id=300 then res else 0 end) cpx_res 
from dm.domain_res res 
left join dm.hierarchies b using (target_id) 
where concat_ws('/', res.year, res.month ,res.day) between '{start_at}' and '{end_at}' 
and res.ssp_id > 0 
and res.tactics_id > 0 
group by 1,2,3,4,5)
select
'全体' kbn,
concat_ws('/', year, month, day) sum_date,
ssp_id parent_ssp_id, 
ads.child_ssp_id,
ads.child_ssp_id display_child_ssp_id,
cast(native_request as int) nativeflg, 
cast(video_request as int) videoflg,
sum(logicad_request_id_1_sum_native_plcmtcnt) req,  
sales, 
res,
imp, 
rta_res,
rta_imp,
rta_sales,
ctv1,
ctv2, 
rta_ctv1,  
rta_ctv2, 
cpx_res,
cpx_imp,
cpx_sales,
cpx_ctv1,
cpx_ctv2
from dm.domain_req ads 
left join res on 
concat_ws('/', ads.year, ads.month, ads.day)=res.sum_date
and ads.ssp_id=res.parent_ssp_id
and ads.child_ssp_id=res.child_ssp_id
and cast(native_request as int)=res.nativeflg
and cast(video_request as int)=res.videoflg
left join imp on 
concat_ws('/', ads.year, ads.month, ads.day)=imp.sum_date
and ads.ssp_id=imp.parent_ssp_id
and ads.child_ssp_id=imp.child_ssp_id
and cast(native_request as int)=imp.nativeflg
and cast(video_request as int)=imp.videoflg
where concat_ws('/', year, month ,day) between '{start_at}' and '{end_at}' 
and ads.ssp_id>0
group by 1,2,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
"""
)
with ImpalaResource(**ircfg) as ir:
    data_ssp_d_s = ir.sql_to_pandas(query)

INFO:ailab_tools.smn.impala_client:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_client:hosts : ['172.16.60.117']
INFO:ailab_tools.smn.impala_client:impala_host : 172.16.60.117
INFO:ailab_tools.smn.impala_client:impala_port : 21050
INFO:ailab_tools.smn.impala_client:impala_user : vmspool
INFO:ailab_tools.smn.impala_client:impala_password : 
INFO:ailab_tools.smn.impala_client:request_pool : adhoc_dm01_pool
INFO:ailab_tools.smn.impala_client:use_ssl : False
INFO:ailab_tools.smn.impala_client:configuration : {'REQUEST_POOL': 'adhoc_dm01_pool'}
INFO:ailab_tools.smn.impala_resource:loaded configuration file: /data1/anaconda3/lib/python3.8/site-packages/ailab_tools/config/ailab_tools_config.py
INFO:ailab_tools.smn.impala_resource:httpfs_host: 172.16.60.42
INFO:ailab_tools.smn.impala_resource:httpfs_port: 14000
INFO:impyla_service:Impala open connection OK. hostname = [172.16.60.117]
INFO:impyla_ser

In [8]:
data_ssp=pd.concat([data_ssp_d,data_ssp_d_s])
data_ssp = data_ssp.sort_values(["kbn","sum_date","parent_ssp_id","child_ssp_id","nativeflg","videoflg"], ascending=True)

In [13]:
# path定義
parent_path = pathlib.Path("__data__").resolve().parent
file_path = parent_path / "data"/ f"monthly_ssp_{start_at}.csv"

data_ssp.to_csv(f"monthly_ssp_20250301.csv", index=False, sep=",")